# Boilerplate

In [1]:
import sys
sys.path.insert(0, 'venv/lib/python3.10/site-packages')
import asyncio

from nwv2_python_wrapper import *
import nwv2_python_wrapper

# Run

List all Python wrappers

In [2]:
[f for f in dir(nwv2_python_wrapper) if not f.startswith('__') and f != 'nwv2_python_wrapper']

['EndpointW',
 'FilterInterface',
 'FilterModeW',
 'PacketSettingsW',
 'PacketW',
 'ProcessUniqueIdW',
 'RequestActionW',
 'TransportCmdW',
 'TransportInterface',
 'TransportNoticeW',
 'TransportRspW',
 'new_transport_interface']

## Transport layer setup

Create the Transport layer and run it in the background.

In [3]:
tiface = await new_transport_interface("0.0.0.0", 12345)
tiface.run()

<Future pending cb=[<builtins.PyDoneCallback object at 0x7f17404038b0>()]>

# Filter layer setup

In [5]:
fiface = FilterInterface(tiface, FilterModeW("client"))

In [7]:
dir(fiface)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'run']